Все умерло, а я дурачок((

In [1]:
#!pip install pymc

In [2]:
#!pip install arviz

In [3]:
pip uninstall numba

Found existing installation: numba 0.58.0
Uninstalling numba-0.58.0:
  Would remove:
    /Users/imachome/anaconda3/bin/numba
    /Users/imachome/anaconda3/lib/python3.11/site-packages/numba-0.58.0.dist-info/*
    /Users/imachome/anaconda3/lib/python3.11/site-packages/numba/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install numba --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 14.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numba
    Found existing installation: numba 0.58.0
    Uninstalling numba-0.58.0:
      Successfully uninstalled numba-0.58.0
Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

import scipy.stats as stats
import pymc as pm
import arviz as az

#sns.set_style("whitegrid")

In [6]:
r_1 = pd.read_csv('')
r_2 = pd.read_csv('')
r_3 = pd.read_csv('')
r_4 = pd.read_csv('')
r_5 = pd.read_csv('')
r_6 = pd.read_csv('')

FileNotFoundError: [Errno 2] No such file or directory: ''

In [ ]:
R = np.concatenate((r_1, r_2, r_3, r_4, r_5, r_6), axis = 0)

In [ ]:
M = len(R)
N = [56, 51, 56, 54, 48, 2]
idx = np.repeat(range(M-1), N)
idx = np.append(idx, 7)
np.random.seed(42)

_, ax = plt.subplots(2, 4, figsize=(10, 5), sharex=True, sharey=True)
ax = np.ravel(ax)
j, k = 0, N
for i in range(6):
    ax[i].scatter(x_m[j:k], y_m[j:k])
    ax[i].set_xlabel(f'x_{i}')
    ax[i].set_ylabel(f'y_{i}', rotation=0, labelpad=15)
    ax[i].set_xlim(6, 15)
    ax[i].set_ylim(7, 17)
    j += N
    k += N
plt.tight_layout()





In [ ]:
with pm.Model() as hierarchical_model:
    # априорные гипер-распределения
    α_μ_tmp = pm.Normal('α_μ_tmp', mu=0, sigma=10)
    α_σ_tmp = pm.HalfNormal('α_σ_tmp', sigma=10)
    β_μ = pm.Normal('β_μ', mu=0, sigma=1)
    β_σ = pm.HalfNormal('β_σ', sigma=1)

    # априорные распеределения
    α_tmp = pm.Normal('α_tmp', mu=α_μ_tmp, sigma=α_σ_tmp, shape=M)
    β = pm.Normal('β', mu=β_μ, sigma=β_σ, shape=M)
    ε = pm.HalfCauchy('ε', 5)   
    ν = pm.Exponential('ν', 1/30)
    
    # апосториорное распределение
    y_pred = pm.StudentT('y_pred', mu=α_tmp[idx] + β[idx] * x_centered,
                         sigma=ε, nu=ν, observed=y_m)
    
    # обратное перемасштабирование
    α = pm.Deterministic('α', α_tmp - β * x_m.mean())
    α_μ = pm.Deterministic('α_μ', α_μ_tmp - β_μ * x_m.mean())
    α_σ = pm.Deterministic('α_sd', α_σ_tmp - β_μ * x_m.mean())

    # трассировка
    trace_hm = pm.sample(3000,tune = 1000)



In [ ]:
_, ax = plt.subplots(2, 4, figsize=(15, 10), sharex=True, sharey=True,
                     constrained_layout=True)
trace_hmm = trace_hm.posterior.stack(samples={"chain", "draw"})

ax = np.ravel(ax)
j, k = 0, N
x_range = np.linspace(x_m.min(), x_m.max(), 10)

for i in range(M):
    ax[i].scatter(x_m[j:k], y_m[j:k])
    ax[i].set_xlabel(f'x_{i}')
    ax[i].set_ylabel(f'y_{i}', labelpad=17, rotation=0)
    alpha_m = trace_hmm['α'].sel({'α_dim_0':i}).mean().item()
    beta_m  = trace_hmm['β'].sel({'β_dim_0':i}).mean().item()
    ax[i].plot(x_range, alpha_m + beta_m * x_range, c='r',
               label=f'y = {alpha_m:.2f} + {beta_m:.2f} * x'+' - иерахич. модель')
    
    ax[i].legend()
    plt.xlim(x_m.min()-1, x_m.max()+1)
    plt.ylim(y_m.min()-1, y_m.max()+1)
    j += N
    k += N

In [ ]:
az.summary(trace_hmm)